<a href="https://colab.research.google.com/github/Daria-Lytvynenko/ML_course/blob/main/HW_2_4_kNN_%D0%9A%D1%80%D0%BE%D1%81%D0%B2%D0%B0%D0%BB%D1%96%D0%B4%D0%B0%D1%86%D1%96%D1%8F_%D1%96_%D1%82%D1%8E%D0%BD%D0%B8%D0%BD%D0%B3_%D0%B3%D1%96%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [28]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, auc, classification_report
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from sklearn.neighbors import KNeighborsClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
sys.path.append('drive/MyDrive/ML_course')

In [4]:
from process_bank_churn import *

In [5]:
train=pd.read_csv('drive/MyDrive/ML_course/train.csv').set_index('id')

In [6]:
result=preprocess_data(train)

In [7]:
X_train=result['X_train']
train_targets=result['train_targets']
X_val=result['X_val']
val_targets=result['val_targets']
input_cols=result['input_cols']
scaler=result['scaler']
encoder=result['encoder']
categorical_cols=result['categorical_cols']
numeric_cols=result['numeric_cols']
categories=result['categories']

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [9]:
knn = KNeighborsClassifier()
knn.fit(X_train, train_targets)
pred_train=knn.predict(X_train)
pred_val=knn.predict(X_val)
pred_proba_train=knn.predict_proba(X_train)[:,1]
pred_proba_val=knn.predict_proba(X_val)[:,1]
fpr_tr, tpr_tr, threshold_tr=roc_curve(train_targets, pred_proba_train)
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba_val)
roc_auc_tr=auc(fpr_tr, tpr_tr)*100
roc_auc_val=auc(fpr_val, tpr_val)*100

In [10]:
roc_auc_tr, roc_auc_val

(96.08380648944805, 87.61994417925472)

Модель поганої якості, дуже перенавчається high variance.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [16]:
knn=KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, train_targets)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [17]:
knn_best = knn_gs.best_estimator_

print(knn_gs.best_params_, knn_gs.best_score_)

{'n_neighbors': 11} 0.8861333333333332


In [20]:
knn.metric

'minkowski'

In [18]:
pred_train=knn_best.predict(X_train)
pred_val=knn_best.predict(X_val)
pred_proba_train=knn_best.predict_proba(X_train)[:,1]
pred_proba_val=knn_best.predict_proba(X_val)[:,1]
fpr_tr, tpr_tr, threshold_tr=roc_curve(train_targets, pred_proba_train)
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba_val)
roc_auc_tr=auc(fpr_tr, tpr_tr)*100
roc_auc_val=auc(fpr_val, tpr_val)*100

In [19]:
roc_auc_tr, roc_auc_val

(94.69868922312693, 90.47212450983531)

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [34]:
tree=DecisionTreeClassifier()
parameters={'max_depth': np.arange(1,20,2), 'max_leaf_nodes': np.arange(2,10,1) }
tree_gs=GridSearchCV(tree, parameters, cv=3, scoring='roc_auc' )
tree_gs.fit(X_train, train_targets)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                         'max_leaf_nodes': array([2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='roc_auc')

In [35]:
dt_best=tree_gs.best_estimator_
print(tree_gs.best_params_, tree_gs.best_score_)

{'max_depth': 5, 'max_leaf_nodes': 9} 0.8948407274686595


In [36]:
pred_train=dt_best.predict(X_train)
pred_val=dt_best.predict(X_val)
pred_proba_train=dt_best.predict_proba(X_train)[:,1]
pred_proba_val=dt_best.predict_proba(X_val)[:,1]
fpr_tr, tpr_tr, threshold_tr=roc_curve(train_targets, pred_proba_train)
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba_val)
roc_auc_tr=auc(fpr_tr, tpr_tr)*100
roc_auc_val=auc(fpr_val, tpr_val)*100

In [37]:
roc_auc_tr, roc_auc_val

(89.96501465088585, 90.01209259346201)

In [38]:
test=pd.read_csv('drive/MyDrive/ML_course/test.csv').set_index('id')
sample_submission=pd.read_csv('drive/MyDrive/ML_course/sample_submission.csv')
test_inputs=preprocess_new_data(test, encoder, scaler, categorical_cols, numeric_cols, categories)
sample_submission['Exited']=dt_best.predict(test_inputs)
sample_submission.set_index('id')
sample_submission.to_csv('drive/MyDrive/ML_course/submission_tree.csv', index=False)

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [39]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [40]:
tree=DecisionTreeClassifier()
tree_rs=RandomizedSearchCV(tree, params_dt, cv=3, random_state=42, scoring='roc_auc')
tree_rs.fit(X_train, train_targets)

RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   random_state=42, scoring='roc_auc')

In [41]:
dt_random_search_best=tree_rs.best_estimator_
print(tree_rs.best_params_, tree_rs.best_score_)

{'splitter': 'random', 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_leaf_nodes': 16, 'max_features': None, 'max_depth': 10, 'criterion': 'gini'} 0.8836909116730238


In [42]:
pred_train=dt_random_search_best.predict(X_train)
pred_val=dt_random_search_best.predict(X_val)
pred_proba_train=dt_random_search_best.predict_proba(X_train)[:,1]
pred_proba_val=dt_random_search_best.predict_proba(X_val)[:,1]
fpr_tr, tpr_tr, threshold_tr=roc_curve(train_targets, pred_proba_train)
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba_val)
roc_auc_tr=auc(fpr_tr, tpr_tr)*100
roc_auc_val=auc(fpr_val, tpr_val)*100

In [45]:
roc_auc_tr, roc_auc_val

(88.19755760228698, 88.02804726993028)

In [46]:
test=pd.read_csv('drive/MyDrive/ML_course/test.csv').set_index('id')
sample_submission=pd.read_csv('drive/MyDrive/ML_course/sample_submission.csv')
test_inputs=preprocess_new_data(test, encoder, scaler, categorical_cols, numeric_cols, categories)
sample_submission['Exited']=dt_random_search_best.predict(test_inputs)
sample_submission.set_index('id')
sample_submission.to_csv('drive/MyDrive/ML_course/submission_tree.csv', index=False)

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉